# Fluidize-Python Interactive Demo

This notebook demonstrates the fluidize-python library for managing scientific computing projects.

## Setup

First, let's import the client and see where our projects will be stored:

In [ ]:
# Import the fluidize client - handlers auto-register!
from fluidize.client import FluidizeClient

# Create client and config
client = FluidizeClient(mode="local")

print(f"📁 Projects will be stored in: {client.config.local_projects_path}")
print(f"📁 Base directory: {client.config.local_base_path}")
print(f"🚀 Client ready in '{client.mode}' mode!")

We now load a project. This contains everything we need to run a simulation pipeline!

In [ ]:
# Get our MUJOCO project for node creation
project = client.projects.get("MUJOCO")

print(f"🎯 Working with project: {project.label}")
print(f"📊 Current graph state: {len(project.graph.get().nodes)} nodes, {len(project.graph.get().edges)} edges")

We now list the parameters that we can tune for this simulation. Here to make it simple, we just have one, which will see in a bit!

In [ ]:
print(project.graph.show_parameters("Mujoco-Simulation"))

## Example Run

We now run the simulation. The payload here is for you to describe what this run was. All the things like parameters and your source code is automatically tracked for each run. 

In [ ]:
from fluidize.core.types.runs import RunFlowPayload

# Just providing information about what the run is 
payload = RunFlowPayload(
    name="simulation-run-1", description="Running with Velocity 20", tags=["simulation", "analysis"]
)


result = project.runs.run_flow(payload)
run_number = result['run_number']

## 4. Looking at Results

In [ ]:
# List all output files from the latest run
node_id = "Mujoco-Simulation"

# Get list of output files
output_files = project.runs.list_node_outputs(run_number, node_id)
print(f"📂 Output files from run {run_number}:")
for file in output_files:
    print(f"  - {file}")

# Get the output path
output_path = project.runs.get_output_path(run_number, node_id)
print(f"\n📁 Output directory: {output_path}")
print(f"Directory exists: {output_path.exists()}")

In [ ]:
# Create a reusable visualization function
from IPython.display import Image, Video, display

def visualize_run_results(project, run_number, node_id):
    """Display all output files from a run"""
    print(f"📊 Results from Run {run_number} - Node: {node_id}")
    
    # Get list of output files
    output_files = project.runs.list_node_outputs(run_number, node_id)
    print(f"📂 Output files:")
    for file in output_files:
        print(f"  - {file}")

    # Get the output path
    output_path = project.runs.get_output_path(run_number, node_id)
    print(f"\n📁 Output directory: {output_path}")
    print(f"Directory exists: {output_path.exists()}")
    
    if not output_files:
        print("❌ No output files found!")
        return
    
    # Look for images and videos
    image_files = [f for f in output_files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.svg'))]
    video_files = [f for f in output_files if f.lower().endswith(('.mp4', '.avi', '.mov', '.webm'))]

    print("\n🖼️  Displaying images:")
    for img_file in image_files:
        img_path = output_path / img_file
        if img_path.exists():
            print(f"📷 {img_file}:")
            display(Image(str(img_path), width=800, height=600))
        else:
            print(f"❌ {img_file} not found")

    print("\n🎥 Displaying videos:")
    for vid_file in video_files:
        vid_path = output_path / vid_file
        if vid_path.exists():
            print(f"🎬 {vid_file}:")
            print(f"   File size: {vid_path.stat().st_size / 1024 / 1024:.2f} MB")
            display(Video(str(vid_path.absolute()), width=600, height=400, embed=True))
        else:
            print(f"❌ {vid_file} not found")

    if not image_files and not video_files:
        print("📝 No image or video files found. Available files:")
        for file in output_files:
            file_path = output_path / file
            if file_path.exists():
                print(f"  📄 {file} ({file_path.stat().st_size} bytes)")

# Test the function with the previous run
visualize_run_results(project, run_number, "Mujoco-Simulation")

## 5. Parameter Experiment - Change Motor Strength and Compare Results

In [ ]:
# Step 1: Change the motor strength parameter from 20.0 to 35.0
from fluidize.core.types.parameters import Parameter

print("🔧 Updating motor strength parameter from 20.0 to 35.0...")

# Create updated parameter
new_motor_strength = Parameter(
    name="motor_strength",
    value="35.0",  # Increased from 20.0 to 35.0
    type="text",
    label="Motor Strength", 
    description="Control signal strength for bat motor (higher = faster swing, more collision force)",
    scope="simulation",
    location=["source/pinata_simulation.py"]
)

# Update the parameter
project.graph.upsert_parameter("Mujoco-Simulation", new_motor_strength)

# Show the updated parameters
print("\n✅ Parameters updated!")
print(project.graph.show_parameters("Mujoco-Simulation"))

In [ ]:
# Step 2: Run a new simulation with the updated parameters
from fluidize.core.types.runs import RunFlowPayload

print("🚀 Starting new simulation with motor strength = 35.0...")

payload = RunFlowPayload(
    name="high-motor-experiment", 
    description="Testing increased motor strength (35.0 vs 20.0)", 
    tags=["experiment", "parameter-study", "high-power"]
)

result = project.runs.run_flow(payload)
new_run_number = result['run_number']

print(f"✅ Simulation started! Run number: {new_run_number}")
print(f"Status: {result['flow_status']}")
print("\n⏳ Wait a moment for simulation to complete, then run the next cell...")

In [ ]:
# Step 3: Visualize the new results using our reusable function
print("📊 Results from the HIGH motor strength simulation (35.0):")
print("="*60)

# Use the new_run_number from the previous cell
visualize_run_results(project, new_run_number, "Mujoco-Simulation")

In [ ]:
# Step 4: Compare with the previous results (motor strength = 20.0)
print("📊 COMPARISON - Results from the NORMAL motor strength simulation (20.0):")
print("="*60)

# Show the previous run results for comparison
visualize_run_results(project, 11, "Mujoco-Simulation")  # Run 11 was with motor_strength=20.0